In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Get the data

In [ ]:
data = np.array([
                    1, 220, 240, 1, 7, 17.5, 
                    2, 220, 300, 1, 8, 20.5,
                    3, 220, 330, 1, 9, 22.5,
                    4, 220, 400, 1, 10, 17.5,
                    5, 220, 500, 1, 11, 20.5,
                    6, 220, 600, 1, 12, 22.5,
                    7, 330, 240, 1, 9, 10.7,
                    8, 330, 300, 1, 10, 17.5,
                    9, 330, 330, 1, 11, 20.5,
                    10, 330, 400, 2, 12, 22.5,
                    11, 330, 500, 2, 13, 25.5,
                    12, 330, 600, 2, 14, 22.5,
                    13, 500, 300, 2, 12, 17, 
                    14, 500, 300, 2, 13, 22, 
                    15, 500, 330, 2, 14, 27,
                    16, 500, 400, 3, 14, 17,
                    17, 500, 500, 3, 15, 22,
                    18, 500, 600, 3, 15, 27,
                    19, 750, 240, 3, 17.5, 28,
                    20, 750, 300, 3, 18, 30,
                    21, 750, 400, 4, 18.5, 32,
                    22, 750, 400, 4, 19, 35, 
                    23, 750, 500, 4, 19.5, 32,
                    24, 750, 500, 4, 20, 35,
                    25, 220, 240, 1, 9, 20.5,
                    26, 330, 300, 1, 12, 22.5,
                    27, 500, 400, 2, 15, 27,
                    28, 750, 500, 3, 18, 30,
                    29, 330, 500, 2, 13, 20.5,
                    30, 500, 600, 2, 14, 22.5
]).reshape(-1, 6)

data = pd.DataFrame(data=data, columns=["Index", "U", "S", "n", "$D_0$", "H"])
data["Index"] = data["Index"].astype(int)
data["n"] = data["n"].astype(int)

data = data.set_index("Index");
data["spg"] = np.zeros(30) + np.nan

data["spg"][data["U"] == 330] = 20;
data["spg"][data["U"] == 500] = 30;
data["spg"][data["U"] == 750] = 40;

In [ ]:
def calculate_E(U, H, D_0, S, x):
    
    pi = np.pi
    eps_0 = 8.85 * 1e-12

    S = S * 1e-6    
    d = np.sqrt(4 * S / pi)
    c = 24e-12/(np.log(2*D_0/d)/np.log(10))
        
    E = (c * U / (2*np.sqrt(3)*pi*eps_0))*(2*H/((x-D_0)**2+H**2) - H/(x**2+H**2) - H/((x+D_0)**2 + H**2))
    
    return E

In [ ]:
result_E = np.zeros((30, 7))
result_E_x_values = [[] for i in range(30)]

for index, (U, S, n, D_0, H, spg) in enumerate(data.to_numpy()):
    result_E[index, 0] = calculate_E(U, H, D_0, S, 0)
    result_E[index, 1] = calculate_E(U, H, D_0, S, D_0)
    result_E[index, 2] = calculate_E(U, H, D_0, S, 2*D_0)
    result_E[index, 3] = calculate_E(U, H, D_0, S, 3*D_0)
    result_E[index, 4] = calculate_E(U, H, D_0, S, 4*D_0)
    result_E[index, 5] = calculate_E(U, H, D_0, S, 5*D_0)
    
    result_E_x_values[index] = [0, D_0, 2*D_0, 3*D_0, 4*D_0, 5*D_0]
    
    if not np.isnan(spg):
        result_E[index, 6] = calculate_E(U, H, D_0, S, spg)
        result_E_x_values[index].append(spg)
    else:
        result_E[index, 6] = np.nan
        
    result_E_x_values[index].sort()

In [ ]:
df_E = pd.DataFrame(data=result_E, columns=["0", "D_0", "2D_0", "3D_0", "4D_0", "5D_0", "ՍՊԳ"])
df_E = df_E.set_index(np.arange(1, 31))

# PLot one example

In [ ]:
U, S, n, D_0, H, spg

In [ ]:
idx = 11
row = data.to_numpy()[idx]
U, S, n, D_0, H, spg = row

one_result = np.zeros(7)
x_values = np.array([0, D_0, 2*D_0, 3*D_0, 4*D_0, 5*D_0, spg])
x_labels = np.array(["0", "$D_0$", "$2D_0$", "$3D_0$", "$4D_0$", "$5D_0$", "ՍՊԳ"])

one_result[0] = calculate_E(U, H, D_0, S, 0)
one_result[1] = calculate_E(U, H, D_0, S, D_0)
one_result[2] = calculate_E(U, H, D_0, S, 2*D_0)
one_result[3] = calculate_E(U, H, D_0, S, 3*D_0)
one_result[4] = calculate_E(U, H, D_0, S, 4*D_0)
one_result[5] = calculate_E(U, H, D_0, S, 5*D_0)
one_result[6] = calculate_E(U, H, D_0, S, spg)

indices = np.argsort(x_values)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x_values[indices], one_result[indices], ls="-.")
ax.scatter(x_values[indices], one_result[indices], color="black")

ax.set_xticks(x_values[indices])
ax.set_xticklabels(x_labels[indices], rotation='horizontal', fontsize=12)

ax.set_xlabel("d, մ", rotation=0, fontsize=15)
ax.set_ylabel("E, կՎ", rotation=0, fontsize=15, labelpad=0, loc="top")
fig.tight_layout()
plt.show()

# Problem N2

In [ ]:
def calculate_L(U, H, D_0, S, n):
    
    eps_0 = 8.85e-12
    pi = np.pi
    
    S = S * 10**-6
    d = np.sqrt(4 * S / pi)
    r = d / 2
    c = 24e-12 / (np.log(2*D_0/d)/np.log(10))

    B = 100
    E_ar = c * U / (2*np.sqrt(3)*pi*eps_0*r)
    L = 20 + 0.0111*E_ar + 900*r + 15*np.log(n) - 20*np.log(B)/np.log(10)
    return L

In [ ]:
resutl_L = []
for U, S, n, D_0, H, _ in data.to_numpy():
    resutl_L.append(calculate_L(U, H, D_0, S, n))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
ax.plot(range(1, 31), resutl_L, ls="-.")
ax.set_xlabel(range(1, 31))
ax.set_xticks(range(1, 31))
ax.scatter(range(1, 31), resutl_L, color="black")

ax.axhline(y=45, ls="--", color="red")
ax.set_xlabel("Տվյալ", rotation=0, fontsize=20)
ax.set_ylabel("Աղմուկ (դԲ)", rotation=45, fontsize=10, labelpad=20)
fig.tight_layout()
plt.show()

In [ ]:
resutl_L[idx]